## Ermittlung Gepris-Personen-ID und Speicherung in einer Liste. Danach mit Hilfe der neuen Personen-Liste alle Projekte downloaden, die zu den jeweiligen Personen gehören. Ergebnisse werden in JSON-Dateien gespeichert.

In [23]:
import requests
from bs4 import BeautifulSoup
import json
import re  # Regex-Modul zum Bereinigen des Dateinamens

def get_gepris_id_for_person(name):
    search_url = "https://gepris.dfg.de/gepris/OCTOPUS"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    params = {
        "context": "person",
        "task": "doSearchSimple",
        "keywords_criterion": name,
        "findButton": "historyCall",
        "hitsPerPage": 10,
        "index": 0,
        "nurProjekteMitAB": False
    }

    try:
        response = requests.get(search_url, params=params, headers=headers)
        response.raise_for_status()  # Raise an exception for bad responses
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the first search result link containing the name
        search_results = soup.find_all("a", href=True)
        for result in search_results:
            if name.lower() in result.text.lower():  # Case-insensitive comparison
                return result['href']

        print(f"No GEPRIS ID found for {name}")
        return None
    
    except requests.exceptions.RequestException as e:
        print(f"Error accessing GEPRIS for {name}: {e}")
        return None

# Liste von Suchbegriffen für Personen
search_terms = ["Förstner, Konrad", "Becker, Anke", "Bork, Peer", "Clavel, Thomas", "Goesmann, Alexander"]

# Liste für die Ergebnisse initialisieren
gepris_id_links = []

# Durchlaufe jeden Suchbegriff und rufe die GEPRIS-ID ab
for person_name in search_terms:
    gepris_id_link = get_gepris_id_for_person(person_name)
    if gepris_id_link:
        # Vervollständige die URL mit dem Basis-Link, falls nötig
        if not gepris_id_link.startswith("https://"):
            gepris_id_link = f"https://gepris.dfg.de{gepris_id_link}"
        gepris_id_links.append(gepris_id_link)
        print(f"GEPRIS ID Link für {person_name}: {gepris_id_link}")
    else:
        print(f"GEPRIS ID Link für {person_name} konnte nicht gefunden werden.")

# Durchlaufe die Liste der GEPRIS-ID-Links
for url in gepris_id_links:
    try:
        # Initialisiere ein leeres Dictionary für die Person und ihre Projekte
        person_data = {}

        # Führe eine HTTP-Anfrage durch, um den HTML-Inhalt der Seite zu erhalten
        response = requests.get(url)

        # Überprüfe, ob die Anfrage erfolgreich war (Status-Code 200)
        if response.status_code == 200:
            # Parsen des HTML-Inhalts mit Beautiful Soup
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extrahiere den Namen der Person (aus dem Titel der Seite)
            person_name = soup.title.text.strip().replace("DFG - GEPRIS - ", "")

            # Bereinige den Personennamen für den Dateinamen (entferne Sonderzeichen und Leerzeichen)
            clean_person_name = re.sub(r'[^\w\s-]', '', person_name).strip()
            clean_person_name = re.sub(r'[-\s]+', '-', clean_person_name)

            # Füge den bereinigten Namen zur Personendaten hinzu
            person_data["Person Name"] = person_name

            # Finde alle Links auf der Seite
            links = soup.find_all('a', href=True)

            # Filtere nur die Links, die auf Projekte verweisen
            project_links = [link for link in links if '/gepris/projekt/' in link['href']]

            # Extrahiere den Text (Namen) und den Link aus den gefundenen Projekt-Links
            projects_data = []
            for project_link in project_links:
                project_name = project_link.get_text(strip=True)
                project_url = project_link['href']
                projects_data.append({
                    "Projekt Name": project_name,
                    "Projekt Link": project_url
                })

            # Füge die Projekte zum Dictionary der Person hinzu
            person_data["Projekte"] = projects_data

            # Erstelle den Dateinamen mit dem bereinigten Personennamen
            output_file = f"{clean_person_name}.json"

            # Speichere die Daten in einer JSON-Datei mit dem entsprechenden Dateinamen
            with open(output_file, 'w', encoding='utf-8') as json_file:
                json.dump(person_data, json_file, ensure_ascii=False, indent=2)

            print(f"Daten für Person '{person_name}' erfolgreich in '{output_file}' gespeichert.")

        else:
            print(f"Fehler bei der Anfrage für {url}: {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"Fehler bei der HTTP-Anfrage für {url}: {e}")
    except Exception as ex:
        print(f"Allgemeiner Fehler für {url}: {ex}")


GEPRIS ID Link für Förstner, Konrad: https://gepris.dfg.de/gepris/person/262159783
GEPRIS ID Link für Becker, Anke: https://gepris.dfg.de/gepris/person/1592887
GEPRIS ID Link für Bork, Peer: https://gepris.dfg.de/gepris/person/1011382
GEPRIS ID Link für Clavel, Thomas: https://gepris.dfg.de/gepris/person/184220620
GEPRIS ID Link für Goesmann, Alexander: https://gepris.dfg.de/gepris/person/188428736
Daten für Person 'Professor Dr. Konrad  Förstner' erfolgreich in 'Professor-Dr-Konrad-Förstner.json' gespeichert.
Daten für Person 'Professorin Dr. Anke  Becker' erfolgreich in 'Professorin-Dr-Anke-Becker.json' gespeichert.
Daten für Person 'Professor Dr. Peer  Bork' erfolgreich in 'Professor-Dr-Peer-Bork.json' gespeichert.
Daten für Person 'Professor Dr. Thomas  Clavel' erfolgreich in 'Professor-Dr-Thomas-Clavel.json' gespeichert.
Daten für Person 'Professor Dr. Alexander  Goesmann' erfolgreich in 'Professor-Dr-Alexander-Goesmann.json' gespeichert.


## alles unter hier drunter ist alt

In [33]:
import requests
from bs4 import BeautifulSoup

## Alle Projekt IDs von Professor Dr. Alexander Goesmann in GEPRIS

In [34]:
# URL der Website
url = 'https://gepris.dfg.de/gepris/person/188428736'

# Führe eine HTTP-Anfrage durch, um den HTML-Inhalt der Seite zu erhalten
response = requests.get(url)

# Überprüfe, ob die Anfrage erfolgreich war (Status-Code 200)
if response.status_code == 200:
    # Parsen des HTML-Inhalts mit Beautiful Soup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Finde alle Links auf der Seite
    links = soup.find_all('a', href=True)

    # Filtere nur die Links, die auf Projekte verweisen
    project_links = [link['href'] for link in links if '/gepris/projekt/' in link['href']]

    # Drucke die gefundenen Projekt-Links
    for project_link in project_links:
        print(project_link)

else:
    print(f"Fehler bei der Anfrage: {response.status_code}")

/gepris/projekt/319835486
/gepris/projekt/447603908
/gepris/projekt/456668568
/gepris/projekt/458957343
/gepris/projekt/325443116
/gepris/projekt/221270173
/gepris/projekt/284237345
/gepris/projekt/183605059
/gepris/projekt/442032008
/gepris/projekt/460129525
/gepris/projekt/255821879
/gepris/projekt/507302435
/gepris/projekt/270041755
/gepris/projekt/491261247


## Alle Namen der Projekte von Professor Dr. Alexander Goesmann in GEPRIS

In [35]:
# URL der Website
url = 'https://gepris.dfg.de/gepris/person/188428736'

# Führe eine HTTP-Anfrage durch, um den HTML-Inhalt der Seite zu erhalten
response = requests.get(url)

# Überprüfe, ob die Anfrage erfolgreich war (Status-Code 200)
if response.status_code == 200:
    # Parsen des HTML-Inhalts mit Beautiful Soup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Finde alle Links auf der Seite
    links = soup.find_all('a', href=True)

    # Filtere nur die Links, die auf Projekte verweisen
    project_links = [link for link in links if '/gepris/projekt/' in link['href']]

    # Extrahiere den Text (Namen) aus den gefundenen Projekt-Links
    project_names = [project_link.get_text(strip=True) for project_link in project_links]

    # Drucke die gefundenen Projektnamen
    for project_name in project_names:
        print(project_name)

else:
    print(f"Fehler bei der Anfrage: {response.status_code}")

Core Unit - Genom Signaturen und integrierte Systembiologie der Pathogen-Wirts Interaktion
Bioinformatische Methoden zur Analyse der mutualistischen RNA-Kommunikation
Bioinformatische Workflows zur skalierbaren Analyse von pflanzlichen “Omics”-Daten in Cloud-Computing-Umgebungen
Evolution von Gennetzwerken: Die Ranunculales als Modellordnung für evolutionäre Innovationen
GRK 2355: Regulatorische Netzwerke im mRNA-Lebenszyklus:
von kodierenden zu nichtkodierenden RNAs
GRK 1906: Informatische Methoden für die Analyse von Genomdiversität und -dynamik
KFO 309: Virus-induced Lung Injury:
Pathobiology and Novel Therapeutic Strategies
Bioinformatische Analyse genomweiter Datensätze
NFDI4BioDiversität – Biodiversität, Ökologie und Umweltdaten
NFDI4Microbiota - Nationale Forschungsdateninfrastruktur für Mikrobiota-Forschung
Genom-weite Untersuchungen zur Artbildung unter sympatrischen Bedingungen in einem marinen Säuger: demographische Entwicklungsgeschichte, Populationsstruktur und lokale Anpa

## Alle Namen und Links der Projekte von Professor Dr. Alexander Goesmann in GEPRIS mit Name des Autors

In [27]:
# URL der Website
url = 'https://gepris.dfg.de/gepris/person/188428736'

# Führe eine HTTP-Anfrage durch, um den HTML-Inhalt der Seite zu erhalten
response = requests.get(url)

# Überprüfe, ob die Anfrage erfolgreich war (Status-Code 200)
if response.status_code == 200:
    # Parsen des HTML-Inhalts mit Beautiful Soup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extrahiere den Namen der Person (aus dem Titel der Seite)
    person_name = soup.title.text.strip()

    # Finde alle Links auf der Seite
    links = soup.find_all('a', href=True)

    # Filtere nur die Links, die auf Projekte verweisen
    project_links = [link for link in links if '/gepris/projekt/' in link['href']]

    # Extrahiere den Text (Namen) und den Link aus den gefundenen Projekt-Links
    for project_link in project_links:
        project_name = project_link.get_text(strip=True)
        project_url = project_link['href']
        print(f"Projekt Name: {project_name}\nLink: {project_url}\n")

    # Drucke den Namen der Person
    print(f"Name der Person: {person_name}")

else:
    print(f"Fehler bei der Anfrage: {response.status_code}")

Projekt Name: Core Unit - Genom Signaturen und integrierte Systembiologie der Pathogen-Wirts Interaktion
Link: /gepris/projekt/319835486

Projekt Name: Bioinformatische Methoden zur Analyse der mutualistischen RNA-Kommunikation
Link: /gepris/projekt/447603908

Projekt Name: Bioinformatische Workflows zur skalierbaren Analyse von pflanzlichen “Omics”-Daten in Cloud-Computing-Umgebungen
Link: /gepris/projekt/456668568

Projekt Name: Evolution von Gennetzwerken: Die Ranunculales als Modellordnung für evolutionäre Innovationen
Link: /gepris/projekt/458957343

Projekt Name: GRK 2355: Regulatorische Netzwerke im mRNA-Lebenszyklus:
von kodierenden zu nichtkodierenden RNAs
Link: /gepris/projekt/325443116

Projekt Name: GRK 1906: Informatische Methoden für die Analyse von Genomdiversität und -dynamik
Link: /gepris/projekt/221270173

Projekt Name: KFO 309: Virus-induced Lung Injury:
Pathobiology and Novel Therapeutic Strategies
Link: /gepris/projekt/284237345

Projekt Name: Bioinformatische Anal

## Erstellen einer Liste aller Ids der 10 Autoren welche in GEPRIS Projekte haben

In [28]:
# kein eintrag haben: jochen blom, Marius Dieckmann, Barbara Götz, Thomas Gübitz, Franziska Hufsky
liste_aller_personen = ["https://gepris.dfg.de/gepris/person/262159783",
                       "https://gepris.dfg.de/gepris/person/1592887",
                       "https://gepris.dfg.de/gepris/person/1011382",
                       "https://gepris.dfg.de/gepris/person/184220620",
                        "https://gepris.dfg.de/gepris/person/188428736",       
                       ]

## Erstellen einer Json Datei mit allen 10 Autoren welche in GEPRIS Projekte haben, mit jeweiligen Projektnamen und Links

In [29]:
import json

In [30]:
# Liste aller Personen-URLs
liste_aller_personen = [
    "https://gepris.dfg.de/gepris/person/262159783",
    "https://gepris.dfg.de/gepris/person/1592887",
    "https://gepris.dfg.de/gepris/person/1011382",
    "https://gepris.dfg.de/gepris/person/184220620",
    "https://gepris.dfg.de/gepris/person/188428736",
]

# Initialisiere eine leere Liste für alle Personen und ihre Projekte
all_persons_data = []

# Durchlaufe die Liste aller Personen
for url in liste_aller_personen:
    # Initialisiere ein leeres Dictionary für die Person und ihre Projekte
    person_data = {}

    # Führe eine HTTP-Anfrage durch, um den HTML-Inhalt der Seite zu erhalten
    response = requests.get(url)

    # Überprüfe, ob die Anfrage erfolgreich war (Status-Code 200)
    if response.status_code == 200:
        # Parsen des HTML-Inhalts mit Beautiful Soup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extrahiere den Namen der Person (aus dem Titel der Seite)
        person_name = soup.title.text.strip().replace("DFG - GEPRIS - ", "")
        person_data["Person Name"] = person_name

        # Finde alle Links auf der Seite
        links = soup.find_all('a', href=True)

        # Filtere nur die Links, die auf Projekte verweisen
        project_links = [link for link in links if '/gepris/projekt/' in link['href']]

        # Extrahiere den Text (Namen) und den Link aus den gefundenen Projekt-Links
        projects_data = []
        for project_link in project_links:
            project_name = project_link.get_text(strip=True)
            project_url = project_link['href']
            projects_data.append({
                "Projekt Name": project_name,
                "Projekt Link": project_url
            })

        # Füge die Projekte zum Dictionary der Person hinzu
        person_data["Projekte"] = projects_data

        # Füge die Person zum Gesamtdictionary hinzu
        all_persons_data.append(person_data)

    else:
        print(f"Fehler bei der Anfrage für {url}: {response.status_code}")

# Speichere alle Daten in einer JSON-Datei
with open('output_all_persons_names_projects_links.json', 'w', encoding='utf-8') as json_file:
    json.dump(all_persons_data, json_file, ensure_ascii=False, indent=2)

# Drucke eine Bestätigungsmeldung
print("Alle Daten erfolgreich in 'output_all_persons_names_projects_links.json' gespeichert.")

Alle Daten erfolgreich in 'output_all_persons_names_projects_links.json' gespeichert.


## Erstellen von Json dateien welche die jeweilige Person und deren Projekte enthalten

In [31]:
import requests
from bs4 import BeautifulSoup
import json
import re  # Regex-Modul zum Bereinigen des Dateinamens

# Liste aller Personen-URLs
liste_aller_personen = [
    "https://gepris.dfg.de/gepris/person/262159783",
    "https://gepris.dfg.de/gepris/person/1592887",
    "https://gepris.dfg.de/gepris/person/1011382",
    "https://gepris.dfg.de/gepris/person/184220620",
    "https://gepris.dfg.de/gepris/person/188428736",
]

# Durchlaufe die Liste aller Personen
for url in liste_aller_personen:
    try:
        # Initialisiere ein leeres Dictionary für die Person und ihre Projekte
        person_data = {}

        # Führe eine HTTP-Anfrage durch, um den HTML-Inhalt der Seite zu erhalten
        response = requests.get(url)

        # Überprüfe, ob die Anfrage erfolgreich war (Status-Code 200)
        if response.status_code == 200:
            # Parsen des HTML-Inhalts mit Beautiful Soup
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extrahiere den Namen der Person (aus dem Titel der Seite)
            person_name = soup.title.text.strip().replace("DFG - GEPRIS - ", "")

            # Bereinige den Personennamen für den Dateinamen (entferne Sonderzeichen und Leerzeichen)
            clean_person_name = re.sub(r'[^\w\s-]', '', person_name).strip()
            clean_person_name = re.sub(r'[-\s]+', '-', clean_person_name)

            # Füge den bereinigten Namen zur Personendaten hinzu
            person_data["Person Name"] = person_name

            # Finde alle Links auf der Seite
            links = soup.find_all('a', href=True)

            # Filtere nur die Links, die auf Projekte verweisen
            project_links = [link for link in links if '/gepris/projekt/' in link['href']]

            # Extrahiere den Text (Namen) und den Link aus den gefundenen Projekt-Links
            projects_data = []
            for project_link in project_links:
                project_name = project_link.get_text(strip=True)
                project_url = project_link['href']
                projects_data.append({
                    "Projekt Name": project_name,
                    "Projekt Link": project_url
                })

            # Füge die Projekte zum Dictionary der Person hinzu
            person_data["Projekte"] = projects_data

            # Erstelle den Dateinamen mit dem bereinigten Personennamen
            output_file = f"{clean_person_name}.json"

            # Speichere die Daten in einer JSON-Datei mit dem entsprechenden Dateinamen
            with open(output_file, 'w', encoding='utf-8') as json_file:
                json.dump(person_data, json_file, ensure_ascii=False, indent=2)

            print(f"Daten für Person '{person_name}' erfolgreich in '{output_file}' gespeichert.")

        else:
            print(f"Fehler bei der Anfrage für {url}: {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"Fehler bei der HTTP-Anfrage für {url}: {e}")
    except Exception as ex:
        print(f"Allgemeiner Fehler für {url}: {ex}")

Daten für Person 'Professor Dr. Konrad  Förstner' erfolgreich in 'Professor-Dr-Konrad-Förstner.json' gespeichert.
Daten für Person 'Professorin Dr. Anke  Becker' erfolgreich in 'Professorin-Dr-Anke-Becker.json' gespeichert.
Daten für Person 'Professor Dr. Peer  Bork' erfolgreich in 'Professor-Dr-Peer-Bork.json' gespeichert.
Daten für Person 'Professor Dr. Thomas  Clavel' erfolgreich in 'Professor-Dr-Thomas-Clavel.json' gespeichert.
Daten für Person 'Professor Dr. Alexander  Goesmann' erfolgreich in 'Professor-Dr-Alexander-Goesmann.json' gespeichert.


## Automatisiert mit Hilfe einer Liste, die Gepris-IDs der Forscher ermitteln

In [32]:
import requests
from bs4 import BeautifulSoup

def get_gepris_id_for_person(name):
    search_url = "https://gepris.dfg.de/gepris/OCTOPUS"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    params = {
        "context": "person",
        "task": "doSearchSimple",
        "keywords_criterion": name,
        "findButton": "historyCall",
        "hitsPerPage": 10,
        "index": 0,
        "nurProjekteMitAB": False
    }

    try:
        response = requests.get(search_url, params=params, headers=headers)
        response.raise_for_status()  # Raise an exception for bad responses
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the first search result link containing the name
        search_results = soup.find_all("a", href=True)
        for result in search_results:
            if name.lower() in result.text.lower():  # Case-insensitive comparison
                return result['href']

        print(f"No GEPRIS ID found for {name}")
        return None
    
    except requests.exceptions.RequestException as e:
        print(f"Error accessing GEPRIS for {name}: {e}")
        return None

# Liste von Suchbegriffen für Personen
search_terms = ["Förstner, Konrad", "Becker, Anke", "Bork, Peer", "Clavel, Thomas", "Goesmann, Alexander"]

# Liste für die Ergebnisse initialisieren
gepris_id_links = []

# Durchlaufe jeden Suchbegriff und rufe die GEPRIS-ID ab
for person_name in search_terms:
    gepris_id_link = get_gepris_id_for_person(person_name)
    if gepris_id_link:
        gepris_id_links.append(gepris_id_link)
        print(f"GEPRIS ID Link für {person_name}: {gepris_id_link}")
    else:
        print(f"GEPRIS ID Link für {person_name} konnte nicht gefunden werden.")

# Ausgabe der gesamten Liste
print("Alle GEPRIS ID Links:")
print(gepris_id_links)





GEPRIS ID Link für Förstner, Konrad: /gepris/person/262159783
GEPRIS ID Link für Becker, Anke: /gepris/person/1592887
GEPRIS ID Link für Bork, Peer: /gepris/person/1011382
GEPRIS ID Link für Clavel, Thomas: /gepris/person/184220620
GEPRIS ID Link für Goesmann, Alexander: /gepris/person/188428736
Alle GEPRIS ID Links:
['/gepris/person/262159783', '/gepris/person/1592887', '/gepris/person/1011382', '/gepris/person/184220620', '/gepris/person/188428736']
